In [ ]:
import mph
import numpy as np
from scipy.optimize import minimize
import time

In [ ]:
class SurfaceTrapOptimizer:
    def __init__(self, model_path):
        print("Starting COMSOL client...")
        self.client = mph.start(cores=1)
        self.model = self.client.load(model_path)
        self.optimization_history = []
        self.failed_simulations = 0
        self.max_failures = 5
        self.cache = {}
        
        # Default parameters from Editable_Param_SurfaceTrap.txt
        self.default_params = {
            'center_width': 0.2, 'rf_width': 0.1, 'rf_spacing': 0.1,
            'V_rf': 250, 'V_dc': 70, 'endcap_rad': 6.0,
            'endcap_thick': 0.5, 'endcap_offset': 1.0, 
            'V_endcap': 1.0, 'f': 10.0, 'length': 1.25
        }
        
        # Baseline metrics from SurfaceTrap_Metrics.txt
        self.baseline_metrics = {
            'depth_eV': 21.07, 'offset_mm': 0.4989, 'P_est_mW': 1.49e-4,
            'trap_x': 0.0, 'trap_y': -0.000475, 'trap_z': 0.0001525
        }
    
    def comprehensive_parameter_sweep(self):
        """
        Comprehensive sweep to understand each parameter's effect on all metrics
        """
        print(" Starting Comprehensive Parameter Sweep...")
        
        param_names = list(self.default_params.keys())
        base_params = list(self.default_params.values())
        
        sweep_results = {}
        metric_ratios = {}
        
        # Extended sweep configuration with wider ranges
        sweep_config = {
            'center_width': {'points': 8, 'range': (0.05, 0.5)},
            'rf_width': {'points': 8, 'range': (0.05, 0.3)},
            'rf_spacing': {'points': 8, 'range': (0.05, 0.3)},
            'V_rf': {'points': 8, 'range': (100, 400)},
            'V_dc': {'points': 8, 'range': (20, 150)},
            'endcap_rad': {'points': 6, 'range': (3, 10)},
            'endcap_thick': {'points': 6, 'range': (0.2, 1.0)},
            'endcap_offset': {'points': 8, 'range': (0.5, 3.0)},
            'V_endcap': {'points': 8, 'range': (0.1, 5.0)},
            'f': {'points': 8, 'range': (5, 30)},
            'length': {'points': 8, 'range': (0.5, 3.0)}
        }
        
        # Sweep each parameter
        for i, param_name in enumerate(param_names):
            print(f"\n📊 Sweeping {param_name}...")
            config = sweep_config[param_name]
            
            if isinstance(config['range'][0], (int, float)):
                values = np.linspace(config['range'][0], config['range'][1], config['points'])
            else:
                base_value = base_params[i]
                low_ratio, high_ratio = config['range']
                values = np.linspace(base_value * low_ratio, base_value * high_ratio, config['points'])
            
            param_results = []
            
            for value in values:
                test_params = base_params.copy()
                test_params[i] = value
                
                metrics = self.safe_simulation(test_params)
                param_results.append({
                    'value': value,
                    'depth_eV': metrics['depth_eV'],
                    'offset_mm': metrics['offset_mm'],
                    'P_est_mW': metrics['P_est_mW'],
                    'trap_x': metrics['trap_x'],
                    'trap_y': metrics['trap_y'],
                    'trap_z': metrics['trap_z']
                })
            
            sweep_results[param_name] = param_results
            
            # Calculate improvement ratios for this parameter
            improvements = self.calculate_metric_improvements(param_results)
            metric_ratios[param_name] = improvements
            
            print(f"  Depth range: {min(r['depth_eV'] for r in param_results):.2f}-{max(r['depth_eV'] for r in param_results):.2f}eV")
            print(f"  Offset range: {min(r['offset_mm'] for r in param_results):.4f}-{max(r['offset_mm'] for r in param_results):.4f}mm")
            print(f"  Power range: {min(r['P_est_mW'] for r in param_results):.2e}-{max(r['P_est_mW'] for r in param_results):.2e}mW")
        
        return sweep_results, metric_ratios
    
    def calculate_metric_improvements(self, param_results):
        """
        Calculate how much each parameter affects each metric
        Returns ratios that show improvement potential
        """
        baseline_depth = self.baseline_metrics['depth_eV']
        baseline_offset = self.baseline_metrics['offset_mm']
        baseline_power = self.baseline_metrics['P_est_mW']
        
        best_depth = min(r['depth_eV'] for r in param_results)
        best_offset = min(r['offset_mm'] for r in param_results)
        best_power = min(r['P_est_mW'] for r in param_results)
        
        # Calculate improvement ratios (lower is better for all metrics)
        depth_improvement = best_depth / baseline_depth  # We want < 1
        offset_improvement = best_offset / baseline_offset  # We want < 1
        power_improvement = best_power / baseline_power  # We want < 1
        
        # Overall improvement potential (harmonic mean of improvements)
        # Lower overall_improvement means better potential
        overall_improvement = 3 / (1/depth_improvement + 1/offset_improvement + 1/power_improvement)
        
        return {
            'depth_ratio': depth_improvement,
            'offset_ratio': offset_improvement, 
            'power_ratio': power_improvement,
            'overall_improvement': overall_improvement,
            'best_depth': best_depth,
            'best_offset': best_offset,
            'best_power': best_power
        }
    
    def analyze_sweep_results(self, sweep_results, metric_ratios):
        """
        Analyze sweep results to find optimal parameter combinations
        """
        print("\n Analyzing Sweep Results...")
        
        # Find parameters that give the best improvements for each metric
        best_depth_params = []
        best_offset_params = [] 
        best_power_params = []
        balanced_params = []
        
        param_names = list(sweep_results.keys())
        
        for param_name in param_names:
            improvements = metric_ratios[param_name]
            param_data = sweep_results[param_name]
            
            # Find the value that gives the best depth
            best_depth_entry = min(param_data, key=lambda x: x['depth_eV'])
            best_depth_params.append({
                'param': param_name,
                'value': best_depth_entry['value'],
                'depth': best_depth_entry['depth_eV'],
                'improvement': improvements['depth_ratio']
            })
            
            # Find the value that gives the best offset
            best_offset_entry = min(param_data, key=lambda x: x['offset_mm'])
            best_offset_params.append({
                'param': param_name, 
                'value': best_offset_entry['value'],
                'offset': best_offset_entry['offset_mm'],
                'improvement': improvements['offset_ratio']
            })
            
            # Find the value that gives the best power
            best_power_entry = min(param_data, key=lambda x: x['P_est_mW'])
            best_power_params.append({
                'param': param_name,
                'value': best_power_entry['value'], 
                'power': best_power_entry['P_est_mW'],
                'improvement': improvements['power_ratio']
            })
            
            # Find balanced improvement
            balanced_entry = min(param_data, key=lambda x: 
                (x['depth_eV']/self.baseline_metrics['depth_eV'] + 
                 x['offset_mm']/self.baseline_metrics['offset_mm'] + 
                 x['P_est_mW']/self.baseline_metrics['P_est_mW']))
            balanced_params.append({
                'param': param_name,
                'value': balanced_entry['value'],
                'depth': balanced_entry['depth_eV'],
                'offset': balanced_entry['offset_mm'],
                'power': balanced_entry['P_est_mW']
            })
        
        # Sort by improvement potential
        best_depth_params.sort(key=lambda x: x['improvement'])
        best_offset_params.sort(key=lambda x: x['improvement']) 
        best_power_params.sort(key=lambda x: x['improvement'])
        
        print("\n Top parameters for DEPTH improvement:")
        for param in best_depth_params[:5]:
            print(f"  {param['param']:15}: {param['value']:8.4f} (depth: {param['depth']:.2f}eV, improvement: {param['improvement']:.3f})")
        
        print("\n Top parameters for OFFSET improvement:")
        for param in best_offset_params[:5]:
            print(f"  {param['param']:15}: {param['value']:8.4f} (offset: {param['offset']:.4f}mm, improvement: {param['improvement']:.3f})")
        
        print("\n Top parameters for POWER improvement:")
        for param in best_power_params[:5]:
            print(f"  {param['param']:15}: {param['value']:8.4f} (power: {param['power']:.2e}mW, improvement: {param['improvement']:.3f})")
        
        # Build candidate starting points
        candidate_points = self.build_candidate_points(best_depth_params, best_offset_params, best_power_params, balanced_params)
        
        return candidate_points
    
    def build_candidate_points(self, best_depth_params, best_offset_params, best_power_params, balanced_params):
        """
        Build multiple candidate starting points based on sweep analysis
        """
        candidates = {}
        param_names = list(self.default_params.keys())
        
        # Candidate 1: Focus on depth reduction (since baseline depth is very high)
        candidates['depth_focused'] = list(self.default_params.values())
        for param_info in best_depth_params[:4]:  # Top 4 depth-improving parameters
            idx = param_names.index(param_info['param'])
            candidates['depth_focused'][idx] = param_info['value']       
        return candidates
    
    def ratio_based_optimization(self, candidate_points, metric_ratios):
        """
        Use metric ratios from sweep to guide optimization
        """
        print("\n Starting Ratio-Based Optimization...")
        
        best_result = None
        best_objective = float('inf')
        
        # Test each candidate point
        for strategy, candidate in candidate_points.items():
            print(f"\n Testing {strategy} strategy...")
            
            # Create customized bounds based on sweep results
            bounds = self.create_custom_bounds(candidate, metric_ratios)
            
            # Custom objective based on metric ratios
            def custom_objective(params):
                return self.ratio_based_objective(params, metric_ratios)
            
            # Optimize from this candidate
            result = minimize(
                custom_objective,
                candidate,
                bounds=bounds,
                method='Nelder-Mead',
                options={'maxiter': 15, 'disp': False}
            )
            
            if result.fun < best_objective:
                best_objective = result.fun
                best_result = result
                print(f"  New best: {result.fun:.4f}")
        
        return best_result
    
    def ratio_based_objective(self, params, metric_ratios):
        """
        Objective function that uses insights from sweep ratios
        """
        metrics = self.safe_simulation(params)
        if metrics is None:
            return 1000.0
        
        depth = metrics['depth_eV']
        offset = metrics['offset_mm'] 
        power = metrics['P_est_mW']
        
        # Target values based on sweep analysis
        target_depth = 0.01  # Much lower than baseline 21.07eV
        target_offset = 0.1  # Better than baseline 0.4989mm
        target_power = 1e-4  # Similar to baseline
        
        # Adaptive weights based on sweep performance
        depth_weight = 0.5
        offset_weight = 0.3
        power_weight = 0.2
        
        # Calculate normalized components
        depth_component = depth_weight * (depth / target_depth)
        offset_component = offset_weight * (offset / target_offset)
        power_component = power_weight * (power / target_power) if power > 0 else 0
        
        objective = depth_component + offset_component + power_component
        
        # Penalties for constraint violations
        if depth < 0.1:  # Minimum trapping depth
            objective += 10.0
        if offset > 1.0:  # Maximum acceptable offset
            objective += 5.0 * offset
        
        self.optimization_history.append({
            'params': params.copy(),
            'metrics': metrics.copy(),
            'objective': objective
        })
        
        print(f"  Depth: {depth:.2f}eV, Offset: {offset:.4f}mm, Power: {power:.2e}mW, Obj: {objective:.3f}")
        
        return objective
    
    def create_custom_bounds(self, candidate, metric_ratios):
        """
        Create smart bounds based on sweep results
        """
        param_names = list(self.default_params.keys())
        bounds = []
        
        for i, param_name in enumerate(param_names):
            current_value = candidate[i]
            
            # Use sweep results to set intelligent bounds
            if param_name in metric_ratios:
                improvement = metric_ratios[param_name]['overall_improvement']
                
                # Parameters with good improvement potential get wider bounds
                if improvement < 0.8:  # Good improvement
                    bounds_range = 0.4  # ±40%
                elif improvement < 1.0:  # Some improvement
                    bounds_range = 0.3  # ±30%
                else:  # No improvement or worse
                    bounds_range = 0.2  # ±20%
            else:
                bounds_range = 0.3  # Default ±30%
            
            # Set parameter-specific minimum bounds
            if param_name in ['center_width', 'rf_width', 'rf_spacing']:
                min_val = max(0.02, current_value * (1 - bounds_range))
                max_val = min(1.0, current_value * (1 + bounds_range))
            elif param_name in ['V_rf', 'V_dc']:
                min_val = max(10, current_value * (1 - bounds_range))
                max_val = min(500, current_value * (1 + bounds_range))
            elif param_name == 'f':
                min_val = max(1, current_value * (1 - bounds_range))
                max_val = min(50, current_value * (1 + bounds_range))
            elif param_name == 'length':
                min_val = max(0.3, current_value * (1 - bounds_range))
                max_val = min(5.0, current_value * (1 + bounds_range))
            else:
                min_val = current_value * (1 - bounds_range)
                max_val = current_value * (1 + bounds_range)
            
            bounds.append((min_val, max_val))
        
        return bounds
    
    def safe_simulation(self, params):
        """Run simulation with error handling"""
        params_key = tuple(params)
        if params_key in self.cache:
            return self.cache[params_key]
        
        try:
            self.update_parameters(params)
            self.model.solve()
            metrics = self.extract_trap_metrics()
            
            if metrics is None:
                raise ValueError("Failed to extract metrics")
            
            self.cache[params_key] = metrics
            self.failed_simulations = 0
            return metrics
            
        except Exception as e:
            print(f"  Simulation failed: {e}")
            self.failed_simulations += 1
            if self.failed_simulations >= self.max_failures:
                raise RuntimeError(f"Too many failures: {self.max_failures}")
            return self.baseline_metrics
    
    def update_parameters(self, params):
        """Update surface trap parameters"""
        parameter_definitions = [
            ('center_width', params[0], 'mm'),
            ('rf_width', params[1], 'mm'),
            ('rf_spacing', params[2], 'mm'),
            ('V_rf', params[3], 'V'),
            ('V_dc', params[4], 'V'),
            ('endcap_rad', params[5], 'mm'),
            ('endcap_thick', params[6], 'mm'),
            ('endcap_offset', params[7], 'mm'),
            ('V_endcap', params[8], 'V'),
            ('f', params[9], 'MHz'),
            ('length', params[10], 'mm')
        ]
        
        for param_name, value, unit in parameter_definitions:
            self.model.parameter(param_name, f'{value}[{unit}]')
    
    def extract_trap_metrics(self):
        """Extract trap metrics"""
        try:
            # Round trap_x to 0 if very small (as suggested)
            trap_x = float(self.model.evaluate('trap_x'))
            if abs(trap_x) < 1e-10:
                trap_x = 0.0
                
            return {
                'depth_eV': float(self.model.evaluate('depth_eV')),
                'offset_mm': float(self.model.evaluate('offset_mm')),
                'P_est_mW': float(self.model.evaluate('P_est_mW')),
                'trap_x': trap_x,
                'trap_y': float(self.model.evaluate('trap_y')),
                'trap_z': float(self.model.evaluate('trap_z'))
            }
        except:
            return None
    
    def get_best_results(self):
        """Get best optimization results"""
        if not self.optimization_history:
            return None, None
        best_entry = min(self.optimization_history, key=lambda x: x['objective'])
        return best_entry['params'], best_entry['metrics']
    
    def export_final_parameters(self, filename='surface_trap_optimized_parameters.txt'):
        """Export optimized parameters"""
        params, metrics = self.get_best_results()
        if params is None:
            print("No results to export")
            return
            
        param_names = list(self.default_params.keys())
        units = ['mm', 'mm', 'mm', 'V', 'V', 'mm', 'mm', 'mm', 'V', 'MHz', 'mm']
        
        with open(filename, 'w') as f:
            for name, value, unit in zip(param_names, params, units):
                f.write(f"{name} {value}[{unit}] \"\"\n")
            
            f.write(f"\n# Performance Metrics:\n")
            f.write(f"# depth_eV: {metrics['depth_eV']:.6f}\n")
            f.write(f"# offset_mm: {metrics['offset_mm']:.6f}\n")
            f.write(f"# P_est_mW: {metrics['P_est_mW']:.6e}\n")
            f.write(f"# trap_x: {metrics['trap_x']:.6e}\n")
            f.write(f"# trap_y: {metrics['trap_y']:.6e}\n")
            f.write(f"# trap_z: {metrics['trap_z']:.6e}\n")
            
            # Improvement from baseline
            f.write(f"\n# Improvement from Baseline:\n")
            depth_imp = ((self.baseline_metrics['depth_eV'] - metrics['depth_eV']) / self.baseline_metrics['depth_eV']) * 100
            offset_imp = ((self.baseline_metrics['offset_mm'] - metrics['offset_mm']) / self.baseline_metrics['offset_mm']) * 100
            power_imp = ((self.baseline_metrics['P_est_mW'] - metrics['P_est_mW']) / self.baseline_metrics['P_est_mW']) * 100
            
            f.write(f"# depth_eV improvement: {depth_imp:+.1f}%\n")
            f.write(f"# offset_mm improvement: {offset_imp:+.1f}%\n") 
            f.write(f"# P_est_mW improvement: {power_imp:+.1f}%\n")
        
        print(f" Optimized parameters exported to {filename}")
        return params, metrics

def main():
    print(" SURFACE TRAP OPTIMIZATION WITH RATIO ANALYSIS")
    print("=" * 60)
    
    start_time = time.time()
    optimizer = SurfaceTrapOptimizer('Surface_trap(v4).mph')
    
    try:
        # Step 1: Comprehensive parameter sweep
        print(" STEP 1: Comprehensive Parameter Sweep")
        print("=" * 50)
        sweep_results, metric_ratios = optimizer.comprehensive_parameter_sweep()
        
        # Step 2: Analyze sweep results and build candidate points
        print("\n STEP 2: Sweep Result Analysis")
        print("=" * 50)
        candidate_points = optimizer.analyze_sweep_results(sweep_results, metric_ratios)
        
        # Step 3: Ratio-based optimization
        print("\n STEP 3: Ratio-Based Optimization")
        print("=" * 50)
        result = optimizer.ratio_based_optimization(candidate_points, metric_ratios)
        
        # Step 4: Export results
        print("\n STEP 4: Exporting Results")
        print("=" * 50)
        optimized_params, optimized_metrics = optimizer.export_final_parameters()
        
        total_time = time.time() - start_time
        total_simulations = len(optimizer.optimization_history)
        
        print("\n" + "="*60)
        print(" SURFACE TRAP OPTIMIZATION COMPLETED!")
        print("="*60)
        print(f"  Total time: {total_time:.1f} seconds")
        print(f" Total simulations: {total_simulations}")
        print(f" Final objective: {result.fun:.4f}")
        
        print(f"\nFINAL RESULTS vs BASELINE:")
        print(f"  Depth:    {optimized_metrics['depth_eV']:.2f} eV (baseline: {optimizer.baseline_metrics['depth_eV']:.2f} eV)")
        print(f"  Offset:   {optimized_metrics['offset_mm']:.4f} mm (baseline: {optimizer.baseline_metrics['offset_mm']:.4f} mm)")
        print(f"  Power:    {optimized_metrics['P_est_mW']:.2e} mW (baseline: {optimizer.baseline_metrics['P_est_mW']:.2e} mW)")
        print(f"  Trap Center: ({optimized_metrics['trap_x']:.3e}, {optimized_metrics['trap_y']:.3e}, {optimized_metrics['trap_z']:.3e}) mm")
        
        # Calculate improvements
        depth_imp = ((optimizer.baseline_metrics['depth_eV'] - optimized_metrics['depth_eV']) / optimizer.baseline_metrics['depth_eV']) * 100
        offset_imp = ((optimizer.baseline_metrics['offset_mm'] - optimized_metrics['offset_mm']) / optimizer.baseline_metrics['offset_mm']) * 100
        power_imp = ((optimizer.baseline_metrics['P_est_mW'] - optimized_metrics['P_est_mW']) / optimizer.baseline_metrics['P_est_mW']) * 100
        
        print(f"\nIMPROVEMENTS:")
        print(f"  Depth:  {depth_imp:+.1f}%")
        print(f"  Offset: {offset_imp:+.1f}%")
        print(f"  Power:  {power_imp:+.1f}%")
        
    except Exception as e:
        print(f"\nOptimization interrupted: {e}")
        try:
            optimizer.export_final_parameters('surface_trap_partial_results.txt')
        except:
            pass
    
    finally:
        print("\nCleaning up...")
        optimizer.client.disconnect()

if __name__ == "__main__":
    main()